[download this notebook here](https://github.com/HumanCompatibleAI/imitation/blob/master/docs/tutorials/5_train_preference_comparisons.ipynb)
# Learning a Reward Function using Preference Comparisons

The preference comparisons algorithm learns a reward function by comparing trajectory segments to each other.

To set up the preference comparisons algorithm, we first need to set up a lot of its internals beforehand:

In [3]:
import random
from imitation.algorithms import preference_comparisons
from imitation.rewards.reward_nets import BasicRewardNet
from imitation.util.networks import RunningNorm
from imitation.util.util import make_vec_env
from imitation.policies.base import FeedForward32Policy, NormalizeFeaturesExtractor
import gymnasium as gym
from stable_baselines3 import PPO
import numpy as np

rng = np.random.default_rng(0)

venv = make_vec_env("Reacher-v4", rng=rng, render_mode='rgb_array', n_envs=1)

reward_net = BasicRewardNet(
    venv.observation_space, venv.action_space, normalize_input_layer=RunningNorm
)

fragmenter = preference_comparisons.RandomFragmenter(
    warning_threshold=0,
    rng=rng,
)
gatherer = preference_comparisons.SyntheticGatherer(rng=rng)
preference_model = preference_comparisons.PreferenceModel(reward_net)
reward_trainer = preference_comparisons.BasicRewardTrainer(
    preference_model=preference_model,
    loss=preference_comparisons.CrossEntropyRewardLoss(),
    epochs=3,
    rng=rng,
)


# Several hyperparameters (reward_epochs, ppo_clip_range, ppo_ent_coef,
# ppo_gae_lambda, ppo_n_epochs, discount_factor, use_sde, sde_sample_freq,
# ppo_lr, exploration_frac, num_iterations, initial_comparison_frac,
# initial_epoch_multiplier, query_schedule) used in this example have been
# approximately fine-tuned to reach a reasonable level of performance.
agent = PPO(
    policy=FeedForward32Policy,
    policy_kwargs=dict(
        features_extractor_class=NormalizeFeaturesExtractor,
        features_extractor_kwargs=dict(normalize_class=RunningNorm),
    ),
    env=venv,
    seed=0,
    n_steps=2048 // venv.num_envs,
    batch_size=64,
    ent_coef=0.01,
    learning_rate=2e-3,
    clip_range=0.1,
    gae_lambda=0.95,
    gamma=0.97,
    n_epochs=10,
)

trajectory_generator = preference_comparisons.AgentTrainerWithVideoBuffering(
    algorithm=agent,
    reward_fn=reward_net,
    venv=venv,
    rng=rng,
    exploration_frac=0.05,
    video_folder='./training_videos',
    video_length=50,
    name_prefix='rl-video'
)


pref_comparisons = preference_comparisons.PreferenceComparisons(
    trajectory_generator,
    reward_net,
    num_iterations=5,  # Set to 60 for better performance
    fragmenter=fragmenter,
    preference_gatherer=gatherer,
    reward_trainer=reward_trainer,
    fragment_length=50,
    transition_oversampling=1,
    initial_comparison_frac=0.1,
    allow_variable_horizon=False,
    initial_epoch_multiplier=4,
    query_schedule="hyperbolic",
)

Then we can start training the reward model. Note that we need to specify the total timesteps that the agent should be trained and how many fragment comparisons should be made.

In [4]:
pref_comparisons.train(
    total_timesteps=50_000,
    total_comparisons=500,
)

Query schedule: [50, 127, 102, 85, 73, 63]
Requested 4750 transitions but only 0 in buffer. Sampling 4750 additional transitions.
Moviepy - Building video /imitation/docs/tutorials/generated_videos/rl-video-step-0-to-step-50.mp4.
Moviepy - Writing video /imitation/docs/tutorials/generated_videos/rl-video-step-0-to-step-50.mp4



Moviepy - Done !
Moviepy - video ready /imitation/docs/tutorials/generated_videos/rl-video-step-0-to-step-50.mp4


Saving video to /imitation/docs/tutorials/generated_videos/rl-video-step-0-to-step-50.mp4
Moviepy - Building video /imitation/docs/tutorials/generated_videos/rl-video-step-0-to-step-50.mp4.
Moviepy - Writing video /imitation/docs/tutorials/generated_videos/rl-video-step-0-to-step-50.mp4



Moviepy - Done !
Moviepy - video ready /imitation/docs/tutorials/generated_videos/rl-video-step-0-to-step-50.mp4


Saving video to /imitation/docs/tutorials/generated_videos/rl-video-step-100-to-step-150.mp4
Moviepy - Building video /imitation/docs/tutorials/generated_videos/rl-video-step-100-to-step-150.mp4.
Moviepy - Writing video /imitation/docs/tutorials/generated_videos/rl-video-step-100-to-step-150.mp4



...

Moviepy - Done !
Moviepy - video ready /imitation/docs/tutorials/generated_videos/rl-video-step-70550-to-step-70600.mp4


Saving video to /imitation/docs/tutorials/generated_videos/rl-video-step-70650-to-step-70700.mp4
Moviepy - Building video /imitation/docs/tutorials/generated_videos/rl-video-step-70650-to-step-70700.mp4.
Moviepy - Writing video /imitation/docs/tutorials/generated_videos/rl-video-step-70650-to-step-70700.mp4



Moviepy - Done !
Moviepy - video ready /imitation/docs/tutorials/generated_videos/rl-video-step-70650-to-step-70700.mp4
------------------------------------------------------
| raw/                                 |             |
|    agent/rollout/ep_len_mean         | 50          |
|    agent/rollout/ep_rew_mean         | -56.3       |
|    agent/rollout/ep_rew_wrapped_mean | -9.21       |
|    agent/time/fps                    | 138         |
|    agent/time/iterations             | 5           |
|    agent/time/time_elapsed           | 74          |
|    agent/time/total_timesteps        | 61440       |
|    agent/train/approx_kl             | 0.011900721 |
|    agent/train/clip_fraction         | 0.355       |
|    agent/train/clip_range            | 0.1         |
|    agent/train/entropy_loss          | -2.37       |
|    agent/train/explained_variance    | 0.966       |
|    agent/train/learning_rate         | 0.002       |
|    agent/train/loss                  | -0.0393     |


-----------------------------------------------------
| mean/                                  |          |
|    agent/rollout/ep_len_mean           | 50       |
|    agent/rollout/ep_rew_mean           | -58.4    |
|    agent/rollout/ep_rew_wrapped_mean   | -9.35    |
|    agent/time/fps                      | 139      |
|    agent/time/iterations               | 3        |
|    agent/time/time_elapsed             | 44       |
|    agent/time/total_timesteps          | 5.73e+04 |
|    agent/train/approx_kl               | 0.0111   |
|    agent/train/clip_fraction           | 0.341    |
|    agent/train/clip_range              | 0.1      |
|    agent/train/entropy_loss            | -2.42    |
|    agent/train/explained_variance      | 0.961    |
|    agent/train/learning_rate           | 0.002    |
|    agent/train/loss                    | -0.0309  |
|    agent/train/n_updates               | 280      |
|    agent/train/policy_gradient_loss    | -0.0104  |
|    agent/train/std        

{'reward_loss': 0.5239280201494694, 'reward_accuracy': 0.7601562514901165}

After we trained the reward network using the preference comparisons algorithm, we can wrap our environment with that learned reward.

In [5]:
from imitation.rewards.reward_wrapper import RewardVecEnvWrapper

learned_reward_venv = RewardVecEnvWrapper(venv, reward_net.predict_processed)

Next, we train an agent that sees only the shaped, learned reward.

In [6]:
learner = PPO(
    seed=0,
    policy=FeedForward32Policy,
    policy_kwargs=dict(
        features_extractor_class=NormalizeFeaturesExtractor,
        features_extractor_kwargs=dict(normalize_class=RunningNorm),
    ),
    env=learned_reward_venv,
    batch_size=64,
    ent_coef=0.01,
    n_epochs=10,
    n_steps=2048 // learned_reward_venv.num_envs,
    clip_range=0.1,
    gae_lambda=0.95,
    gamma=0.97,
    learning_rate=2e-3,
)
learner.learn(100_000)  # Note: set to 100_000 to train a proficient expert

Then we can evaluate it using the original reward.

In [7]:
from stable_baselines3.common.evaluation import evaluate_policy

n_eval_episodes = 100
reward_mean, reward_std = evaluate_policy(learner.policy, venv, n_eval_episodes)
reward_stderr = reward_std / np.sqrt(n_eval_episodes)
print(f"Reward: {reward_mean:.0f} +/- {reward_stderr:.0f}")

Reward: -11 +/- 0


In [8]:
learner.save('imitation_ppo')

In [9]:
from gymnasium.wrappers import RecordVideo

# Create the environment
env = gym.make("Reacher-v4", render_mode='rgb_array')
env = RecordVideo(env, './evaluation_videos', name_prefix="reacher", episode_trigger=lambda x: x % 1 == 0) 

# Run the model in the environment
obs, info = env.reset()
for _ in range(1000):
        action, _states = learner.predict(obs, deterministic=True)
        obs, reward, _ ,done, info = env.step(action)
        if done:
            obs, info = env.reset()
            

env.close()

/venv/lib/python3.8/site-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /imitation/docs/tutorials/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Moviepy - Building video /imitation/docs/tutorials/videos/training-episode-0.mp4.
Moviepy - Writing video /imitation/docs/tutorials/videos/training-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /imitation/docs/tutorials/videos/training-episode-0.mp4
Moviepy - Building video /imitation/docs/tutorials/videos/training-episode-1.mp4.
Moviepy - Writing video /imitation/docs/tutorials/videos/training-episode-1.mp4



Moviepy - Done !
Moviepy - video ready /imitation/docs/tutorials/videos/training-episode-1.mp4
Moviepy - Building video /imitation/docs/tutorials/videos/training-episode-2.mp4.
Moviepy - Writing video /imitation/docs/tutorials/videos/training-episode-2.mp4



Moviepy - Done !
Moviepy - video ready /imitation/docs/tutorials/videos/training-episode-2.mp4
Moviepy - Building video /imitation/docs/tutorials/videos/training-episode-3.mp4.
Moviepy - Writing video /imitation/docs/tutorials/videos/training-episode-3.mp4



...

Moviepy - Done !
Moviepy - video ready /imitation/docs/tutorials/videos/training-episode-16.mp4
Moviepy - Building video /imitation/docs/tutorials/videos/training-episode-17.mp4.
Moviepy - Writing video /imitation/docs/tutorials/videos/training-episode-17.mp4



Moviepy - Done !
Moviepy - video ready /imitation/docs/tutorials/videos/training-episode-17.mp4
Moviepy - Building video /imitation/docs/tutorials/videos/training-episode-18.mp4.
Moviepy - Writing video /imitation/docs/tutorials/videos/training-episode-18.mp4



Moviepy - Done !
Moviepy - video ready /imitation/docs/tutorials/videos/training-episode-18.mp4
Moviepy - Building video /imitation/docs/tutorials/videos/training-episode-19.mp4.
Moviepy - Writing video /imitation/docs/tutorials/videos/training-episode-19.mp4



Moviepy - Done !
Moviepy - video ready /imitation/docs/tutorials/videos/training-episode-19.mp4
Moviepy - Building video /imitation/docs/tutorials/videos/training-episode-20.mp4.
Moviepy - Writing video /imitation/docs/tutorials/videos/training-episode-20.mp4



Moviepy - Done !
Moviepy - video ready /imitation/docs/tutorials/videos/training-episode-20.mp4
